# Kelompok 2
Laili Rofi'ah (20/463604/TK/51596)

Auletta Khansa (20/456359/TK/50489)

Annisa Raihana C P (20/460540/TK/51129)

Natasha Astia Dewanti (20/463610/TK/51602)

Muhammad Rafa Fadlurahman (20/456848/TK/50672)

Rasyid Baskoro Agung (20/456850/TK/50674)

In [44]:
#import library
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.utils.extmath import density
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_confusion_matrix

**FAKE NEWS DATA**

In [45]:
#import fake news dataset
url = 'https://drive.google.com/file/d/15Wtqoyv8zhZ56l3tkxLGgHxsJILwpqQ0/view?usp=sharing'
file_assessment = url.split('/')[-2]
url_assessment = 'https://drive.google.com/uc?id=' + file_assessment
fake = pd.read_csv(url_assessment)

display(fake.head())
print('\n')
display(fake.info())

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


None

**REAL NEWS DATA**

In [46]:
#import real news dataset
url = 'https://drive.google.com/file/d/1z1ZEFPXlEJSsVuoixIhwqzu8plYbQTm0/view?usp=sharing'
file_assessment = url.split('/')[-2]
url_assessment = 'https://drive.google.com/uc?id=' + file_assessment
real = pd.read_csv(url_assessment)

display(real.head())
print('\n')
display(real.info())

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


None

**PROSES TRANSFORM DATA**

In [47]:
display(fake.subject.value_counts())
print('\n')
display(real.subject.value_counts())

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [48]:
#dataset labelling
fake['label'] = 'fake'
real['label'] = 'real'

In [49]:
data = pd.concat([fake, real], axis=0)
data = data.sample(frac=1).reset_index(drop=True)
data.drop('subject', axis=1) #drop subject column

,title,text,date,label
0,ABSOLUTE SUBMISSION: Trump Bows to Neocon Orth...,Consortium News Exclusive: In his Mideast trip...,"June 1, 2017",fake
1,CROOKED HILLARY CLINTON’S Latest Speech To Be ...,Hillary Clinton always goes back to the black ...,"May 19, 2017",fake
2,LIBERAL RAGS LIKE USA TODAY Working Overtime T...,IF YOU HAVEN T FIGURED IT OUT YET THE LIBERAL ...,"Sep 30, 2016",fake
3,Who collects costs for Trump's Taiwan call?,BEIJING (Reuters) - There are many pressure po...,"December 5, 2016",real
4,Episode #120 – SUNDAY WIRE: ‘Crisis of Liberty...,Episode #120 of SUNDAY WIRE SHOW finally resum...,"January 31, 2016",fake
...,...,...,...,...
44893,South Africa's Zuma seeks to appeal ruling on ...,JOHANNESBURG (Reuters) - South Africa s Presid...,"December 22, 2017",real
44894,Trump Jr. Offers Up Revolting Praise Of Dad: ...,"Don Trump Jr., the privileged, wealthy son of ...","February 24, 2016",fake
44895,Brazil Supreme Court sends new Temer graft cha...,BRASILIA (Reuters) - A majority of the judges ...,"September 20, 2017",real
44896,Trump to Republican senators: Don't leave town...,WASHINGTON (Reuters) - U.S. President Donald T...,"July 19, 2017",real


**PREDICTIVE MODELLING**

In [50]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.25)
display(X_train.head())
print('\n')
display(y_train.head())

print("\nThere are {} documents in the training data.".format(len(X_train)))

23243    Ohio Governor John Kasich, who signed a pledge...
25131                                                     
29748    BERLIN (Reuters) - Three of Germany s main par...
12689    WASHINGTON (Reuters) - A U.S. House of Represe...
16197    CANFIELD, Ohio/CLEVELAND (Reuters) - Donald Tr...
Name: text, dtype: object

23243    fake
25131    fake
29748    real
12689    real
16197    real
Name: label, dtype: object


There are 33673 documents in the training data.


In [51]:
#Feature extraction
my_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

# fit the vectorizer and transform X_train into a tf-idf matrix,
# then use the same vectorizer to transform X_test
tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.transform(X_test)

tfidf_train

<33673x108448 sparse matrix of type '<class 'numpy.float64'>'
	with 5141382 stored elements in Compressed Sparse Row format>

In [52]:
#PA Classifier
from sklearn.linear_model import PassiveAggressiveClassifier

pa_clf = PassiveAggressiveClassifier(max_iter=50)
pa_clf.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [53]:
#Implementation of PA Classifier to the dataset
y_pred = pa_clf.predict(tfidf_test)

conf_mat = confusion_matrix(y_test, y_pred)
#plot_confusion_matrix(conf_mat, show_normed=True, colorbar=True, class_names=['Fake', 'Real'])

accscore = accuracy_score(y_test, y_pred)
f1score = f1_score(y_test,y_pred,pos_label='real')

print('The accuracy of prediction is {:.2f}%.\n'.format(accscore*100))
print('The F1 score is {:.3f}.\n'.format(f1score))

The accuracy of prediction is 99.39%.

The F1 score is 0.993.

